# Notebook de mejora

Este notebook es un ejemplo cómo se puede mejorar los hiperparámetros de un algoritmo, y medir la importancia de atributos https://www.kaggle.com/c/house-prices-advanced-regression-techniques/ de la práctica 3 de Inteligencia de Negocio. 

Incluyo el código del notebook anterior, no se explicarán.

Es un ejemplo que es claramente mejorable, se deja a el/la estudiante el mejorarlo para obtener mejores resultados.

In [2]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, RobustScaler


Cambiamos preprocesamiento, lo hacemos con los dos conjuntos juntos.

In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", na_values="NaN") # Definimos na_values para identificar bien los valores perdidos

if 'Id' in train:
    train.drop('Id', axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

test = pd.read_csv("test.csv", na_values="NaN")
test_ids = test.Id
test = test.drop('Id', axis=1)



In [4]:
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [5]:
test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


Vamos a tratar los valores perdidos

In [6]:
# Calcular el porcentaje de valores perdidos en el conjunto de entrenamiento
train_missing_percent = (train.isnull().sum() / len(train)) * 100

# Calcular el porcentaje de valores perdidos en el conjunto de prueba
test_missing_percent = (test.isnull().sum() / len(test)) * 100

# Filtrar las variables con porcentaje mayor que 0 en el conjunto de entrenamiento
train_missing_variables = train_missing_percent[train_missing_percent > 0]

# Filtrar las variables con porcentaje mayor que 0 en el conjunto de prueba
test_missing_variables = test_missing_percent[test_missing_percent > 0]

# Mostrar el porcentaje de valores perdidos en cada variable
print("Porcentaje de valores perdidos en el conjunto de entrenamiento:")
print(train_missing_variables)

print("\nPorcentaje de valores perdidos en el conjunto de prueba:")
print(test_missing_variables)

Porcentaje de valores perdidos en el conjunto de entrenamiento:
LotFrontage     17.739726
Alley           93.767123
MasVnrType       0.547945
MasVnrArea       0.547945
BsmtQual         2.534247
BsmtCond         2.534247
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtFinType2     2.602740
Electrical       0.068493
FireplaceQu     47.260274
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
PoolQC          99.520548
Fence           80.753425
MiscFeature     96.301370
dtype: float64

Porcentaje de valores perdidos en el conjunto de prueba:
MSZoning         0.274160
LotFrontage     15.558602
Alley           92.666210
Utilities        0.137080
Exterior1st      0.068540
Exterior2nd      0.068540
MasVnrType       1.096642
MasVnrArea       1.028101
BsmtQual         3.015764
BsmtCond         3.084304
BsmtExposure     3.015764
BsmtFinType1     2.878684
BsmtFinSF1       0.068540
BsmtFinType2     2.878684
BsmtF

Hay 5 variables con más del 45% de valores perdidos. Vamos a eliminar esas variables del problema.

In [7]:
# Definir el umbral de porcentaje para eliminar variables (en este caso, 45%)
umbral = 45

# Filtrar las variables con porcentaje mayor que el umbral en el conjunto de entrenamiento
variables_a_eliminar_train = train_missing_percent[train_missing_percent > umbral]

# Filtrar las variables con porcentaje mayor que el umbral en el conjunto de prueba
variables_a_eliminar_test = test_missing_percent[test_missing_percent > umbral]

# Eliminar las variables con alto porcentaje de valores perdidos en ambos conjuntos
train.drop(variables_a_eliminar_train.index, axis=1, inplace=True)
test.drop(variables_a_eliminar_test.index, axis=1, inplace=True)

# Mostrar las variables eliminadas
print("Variables eliminadas en el conjunto de entrenamiento:")
print(variables_a_eliminar_train)

print("\nVariables eliminadas en el conjunto de prueba:")
print(variables_a_eliminar_test)

Variables eliminadas en el conjunto de entrenamiento:
Alley          93.767123
FireplaceQu    47.260274
PoolQC         99.520548
Fence          80.753425
MiscFeature    96.301370
dtype: float64

Variables eliminadas en el conjunto de prueba:
Alley          92.666210
FireplaceQu    50.034270
PoolQC         99.794380
Fence          80.123372
MiscFeature    96.504455
dtype: float64


In [8]:
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


Imputamos los valores perdidos

In [9]:
train['is_train'] = 1
test['is_train'] = 0


input_all = pd.concat([train, test])

col_cat = list(input_all.select_dtypes(exclude=np.number).columns)

from sklearn.impute import SimpleImputer

# Valores categóricos por el más frecuente
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit_transform(input_all[col_cat])


# Valores numéricos por KNN
col_num = list(train.select_dtypes(include=np.number).columns)
col_num.remove('SalePrice')

# Configuración del imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos según tus necesidades

# Imputación para el conjunto de entrenamiento
input_all[col_num] = knn_imputer.fit_transform(input_all[col_num])


# Split the data back into train and test sets
train = input_all[input_all['is_train'] == 1].drop(['is_train'], axis=1)
test = input_all[input_all['is_train'] == 0].drop(['is_train'], axis=1)

from sklearn.preprocessing import LabelEncoder
labelers = {}
test_l = test.copy()
train_l = train.copy()


y_train = train_l.SalePrice
X_train = train_l.drop('SalePrice', axis=1)

if 'Id' in test_l:
    test_l.drop('Id', axis=1, inplace=True)

X_test = test_l

In [10]:
# Calcular el porcentaje de valores perdidos en el conjunto de entrenamiento
train_missing_percent = (X_train.isnull().sum() / len(train)) * 100

# Calcular el porcentaje de valores perdidos en el conjunto de prueba
test_missing_percent = (X_test.isnull().sum() / len(test)) * 100

# Filtrar las variables con porcentaje mayor que 0 en el conjunto de entrenamiento
train_missing_variables = train_missing_percent[train_missing_percent > 0]

# Filtrar las variables con porcentaje mayor que 0 en el conjunto de prueba
test_missing_variables = test_missing_percent[test_missing_percent > 0]

# Mostrar el porcentaje de valores perdidos en cada variable
print("Porcentaje de valores perdidos en el conjunto de entrenamiento:")
print(train_missing_variables)

print("\nPorcentaje de valores perdidos en el conjunto de prueba:")
print(test_missing_variables)

Porcentaje de valores perdidos en el conjunto de entrenamiento:
MasVnrType      0.547945
BsmtQual        2.534247
BsmtCond        2.534247
BsmtExposure    2.602740
BsmtFinType1    2.534247
BsmtFinType2    2.602740
Electrical      0.068493
GarageType      5.547945
GarageFinish    5.547945
GarageQual      5.547945
GarageCond      5.547945
dtype: float64

Porcentaje de valores perdidos en el conjunto de prueba:
MSZoning          0.274160
Utilities         0.137080
Exterior1st       0.068540
Exterior2nd       0.068540
MasVnrType        1.096642
BsmtQual          3.015764
BsmtCond          3.084304
BsmtExposure      3.015764
BsmtFinType1      2.878684
BsmtFinType2      2.878684
KitchenQual       0.068540
Functional        0.137080
GarageType        5.209047
GarageFinish      5.346127
GarageQual        5.346127
GarageCond        5.346127
SaleType          0.068540
SalePrice       100.000000
dtype: float64


In [11]:
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60.0,RL,65.0,8450.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,WD,Normal
1,20.0,RL,80.0,9600.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,WD,Normal
2,60.0,RL,68.0,11250.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,WD,Normal
3,70.0,RL,60.0,9550.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,WD,Abnorml
4,60.0,RL,84.0,14260.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,WD,Normal
1456,20.0,RL,85.0,13175.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,WD,Normal
1457,70.0,RL,66.0,9042.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60.0,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,WD,Normal
1458,20.0,RL,68.0,9717.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,WD,Normal


In [12]:
y_train

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1455    175000.0
1456    210000.0
1457    266500.0
1458    142125.0
1459    147500.0
Name: SalePrice, Length: 1460, dtype: float64

In [13]:
X_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20.0,RH,80.0,11622.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,120.0,0.0,0.0,6.0,2010.0,WD,Normal,NaN
1,20.0,RL,81.0,14267.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0.0,0.0,0.0,0.0,12500.0,6.0,2010.0,WD,Normal,NaN
2,60.0,RL,74.0,13830.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,3.0,2010.0,WD,Normal,NaN
3,60.0,RL,78.0,9978.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,6.0,2010.0,WD,Normal,NaN
4,120.0,RL,43.0,5005.0,Pave,IR1,HLS,AllPub,Inside,Gtl,...,0.0,0.0,144.0,0.0,0.0,1.0,2010.0,WD,Normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160.0,RM,21.0,1936.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,6.0,2006.0,WD,Normal,NaN
1455,160.0,RM,21.0,1894.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,4.0,2006.0,WD,Abnorml,NaN
1456,20.0,RL,160.0,20000.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,WD,Abnorml,NaN
1457,85.0,RL,62.0,10441.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,700.0,7.0,2006.0,WD,Normal,NaN


In [14]:
X_train['is_train'] = 'SI'
X_test['is_train'] = 'NO'

input_all = pd.concat([X_train, X_test])


In [15]:
from sklearn.preprocessing import StandardScaler

# Obtener las columnas numéricas
col_num = list(input_all.select_dtypes(include=np.number).columns)

# Crear una instancia del escalador estándar
scaler = StandardScaler()

# Escalar las variables numéricas en input_all
input_all[col_num] = scaler.fit_transform(input_all[col_num])

C:\Users\germa\anaconda3\lib\site-packages\sklearn\utils\extmath.py:1051: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\germa\anaconda3\lib\site-packages\sklearn\utils\extmath.py:1056: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\germa\anaconda3\lib\site-packages\sklearn\utils\extmath.py:1076: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [16]:
input_all

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,is_train,SalePrice
0,0.067331,RL,-0.221432,-0.217879,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,-1.552184,0.157646,WD,Normal,SI,NaN
1,-0.873616,RL,0.442800,-0.072044,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,-0.446925,-0.602962,WD,Normal,SI,NaN
2,0.067331,RL,-0.088585,0.137197,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,1.026753,0.157646,WD,Normal,SI,NaN
3,0.302568,RL,-0.442842,-0.078385,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,-1.552184,-1.363569,WD,Abnorml,SI,NaN
4,0.067331,RL,0.619928,0.518903,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,2.132012,0.157646,WD,Normal,SI,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2.419700,RM,-2.169843,-1.043937,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,-0.078505,-1.363569,WD,Normal,NO,NaN
1455,2.419700,RM,-2.169843,-1.049263,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,-0.815344,-1.363569,WD,Abnorml,NO,NaN
1456,-0.873616,RL,3.985366,1.246808,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,-0.103331,-0.285935,-0.06315,-0.089592,1.026753,-1.363569,WD,Abnorml,NO,NaN
1457,0.655424,RL,-0.354278,0.034605,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,-0.103331,-0.285935,-0.06315,1.144312,0.289914,-1.363569,WD,Normal,NO,NaN


In [17]:
import seaborn as sns

# Calcular la matriz de correlación de las variables numéricas en input_all
correlation_matrix = np.abs(input_all[col_num].corr())

cm = sns.light_palette("green", as_cmap=True)

# Mostrar la matriz de correlación
print("Matriz de correlación de variables numéricas:")
correlation_matrix.style.background_gradient(cmap=cm)

Matriz de correlación de variables numéricas:


C:\Users\germa\anaconda3\lib\site-packages\pandas\io\formats\style.py:3554: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\germa\anaconda3\lib\site-packages\pandas\io\formats\style.py:3555: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
MSSubClass,1.000000,0.415725,0.201730,0.033638,0.065625,0.034409,0.043315,0.004647,0.064060,0.072492,0.126118,0.219821,0.248641,0.309309,0.026482,0.071677,0.010209,0.001728,0.139140,0.178750,0.008796,0.260155,0.040509,0.055151,0.070858,0.046564,0.103358,0.017654,0.015923,0.020867,0.037529,0.049181,0.003080,0.028867,0.001231,0.015028,nan
LotFrontage,0.415725,1.000000,0.448614,0.199229,0.048869,0.103708,0.067597,0.206606,0.213471,0.062840,0.103263,0.347780,0.450150,0.038180,0.005757,0.381892,0.101996,0.006345,0.185114,0.048369,0.248216,0.006434,0.347484,0.272155,0.068143,0.301326,0.355056,0.137716,0.154927,0.015161,0.035786,0.068048,0.161614,0.037220,0.016276,0.009809,nan
LotArea,0.201730,0.448614,1.000000,0.100541,0.035617,0.024128,0.021612,0.125392,0.194145,0.084072,0.021271,0.254187,0.332460,0.031515,0.000554,0.284519,0.127906,0.025838,0.125826,0.034244,0.132801,0.020854,0.213802,0.261185,0.006323,0.180415,0.213232,0.158045,0.104797,0.020974,0.015995,0.054375,0.093708,0.069029,0.004156,0.024234,nan
OverallQual,0.033638,0.199229,0.100541,1.000000,0.093847,0.597554,0.571532,0.433325,0.282074,0.042707,0.274814,0.549261,0.479152,0.245596,0.048393,0.575126,0.164877,0.040519,0.528483,0.272668,0.073075,0.159325,0.389761,0.390753,0.573210,0.600591,0.564973,0.255317,0.298084,0.139256,0.018715,0.042910,0.030740,0.005562,0.030405,0.019614,nan
OverallCond,0.065625,0.048869,0.035617,0.093847,1.000000,0.368477,0.047654,0.136257,0.050746,0.041448,0.137942,0.174110,0.157418,0.005494,0.009048,0.116569,0.042491,0.083911,0.215504,0.088577,0.008477,0.086700,0.092027,0.030999,0.289472,0.181739,0.154100,0.020123,0.068978,0.071044,0.043739,0.043713,0.016876,0.033956,0.006256,0.030102,nan
YearBuilt,0.034409,0.103708,0.024128,0.597554,0.368477,1.000000,0.612235,0.313823,0.279743,0.027565,0.130313,0.408561,0.310814,0.017588,0.144191,0.242666,0.211767,0.030171,0.471169,0.269743,0.053101,0.137614,0.114280,0.170680,0.818560,0.537663,0.480347,0.229426,0.198554,0.374073,0.015958,0.041046,0.002304,0.010886,0.013938,0.012344,nan
YearRemodAdd,0.043315,0.067597,0.021612,0.571532,0.047654,0.612235,1.000000,0.197408,0.152483,0.062072,0.164819,0.298168,0.242245,0.158985,0.060371,0.316972,0.135312,0.046167,0.457980,0.211430,0.021912,0.142431,0.198250,0.134157,0.635907,0.426053,0.376802,0.218513,0.242182,0.220456,0.037433,0.046878,0.011407,0.003124,0.017693,0.033203,nan
MasVnrArea,0.004647,0.206606,0.125392,0.433325,0.136257,0.313823,0.197408,1.000000,0.305241,0.016351,0.088578,0.397604,0.395406,0.120049,0.057960,0.402960,0.143184,0.014718,0.258252,0.192402,0.076814,0.051865,0.277623,0.275631,0.263277,0.362093,0.375043,0.166390,0.148001,0.110084,0.013441,0.064673,0.004415,0.044609,0.001710,0.019253,nan
BsmtFinSF1,0.064060,0.213471,0.194145,0.282074,0.050746,0.279743,0.152483,0.305241,1.000000,0.055014,0.477465,0.536497,0.458189,0.162055,0.065993,0.211917,0.638842,0.078489,0.081779,0.007106,0.113232,0.086288,0.052535,0.293264,0.211520,0.255701,0.310632,0.223647,0.124329,0.099602,0.050931,0.096888,0.084470,0.093309,0.000721,0.022511,nan
BsmtFinSF2,0.072492,0.062840,0.084072,0.042707,0.041448,0.027565,0.062072,0.016351,0.055014,1.000000,0.238247,0.089419,0.084345,0.097709,0.004920,0.017829,0.162845,0.099514,0.075385,0.032421,0.031183,0.037772,0.048353,0.065645,0.055987,0.014822,0.003136,0.098413,0.005853,0.032750,0.023275,0.063310,0.044526,0.005136,0.009566,0.008878,nan


GarageCars y GarageArea están muy correlacionadas, por lo que eliminamos una de ellas. Eliminamos GarageArea porque en el segundo código de prueba, vi que GarageCars tiene más importancia que GarageArea en RandomForest.

In [18]:
# Eliminar la variable 'GarageArea' de input_all
input_all.drop('GarageArea', axis=1, inplace=True)

col_num.remove('GarageArea')

In [19]:
# One-Hot Encoding
input_all = pd.get_dummies(input_all, columns=col_cat, drop_first=True)

# Escalamos logaritmicamente la variable objetivo
y_train = np.log1p(y_train)

In [20]:
# Split the data back into train and test sets
train = input_all[input_all['is_train'] == 'SI'].drop(['is_train'], axis=1)
test = input_all[input_all['is_train'] == 'NO'].drop(['is_train'], axis=1)

from sklearn.preprocessing import LabelEncoder
labelers = {}
test_l = test.copy()
train_l = train.copy()

X_train = train_l.drop('SalePrice', axis=1)

if 'Id' in test_l:
    test_l.drop('Id', axis=1, inplace=True)

X_test = test_l

In [21]:
input_all

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.067331,-0.221432,-0.217879,0.646183,-0.507284,1.046258,0.896833,0.522917,0.581115,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1,-0.873616,0.442800,-0.072044,-0.063185,2.188279,0.154764,-0.395604,-0.572740,1.178257,-0.293098,...,0,0,0,0,1,0,0,0,1,0
2,0.067331,-0.088585,0.137197,0.646183,-0.507284,0.980221,0.848965,0.332854,0.098133,-0.293098,...,0,0,0,0,1,0,0,0,1,0
3,0.302568,-0.442842,-0.078385,0.646183,-0.507284,-1.859351,-0.682812,-0.572740,-0.494618,-0.293098,...,0,0,0,0,1,0,0,0,0,0
4,0.067331,0.619928,0.518903,1.355551,-0.507284,0.947203,0.753229,1.383791,0.469151,-0.293098,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2.419700,-2.169843,-1.043937,-1.481920,1.289758,-0.043346,-0.682812,-0.572740,-0.968819,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1455,2.419700,-2.169843,-1.049263,-1.481920,-0.507284,-0.043346,-0.682812,-0.572740,-0.415585,-0.293098,...,0,0,0,0,1,0,0,0,0,0
1456,-0.873616,3.985366,1.246808,-0.772552,1.289758,-0.373528,0.561757,-0.572740,1.718319,-0.293098,...,0,0,0,0,1,0,0,0,0,0
1457,0.655424,-0.354278,0.034605,-0.772552,-0.507284,0.683057,0.370284,-0.572740,-0.228978,-0.293098,...,0,0,0,0,1,0,0,0,1,0


In [22]:
input_all[input_all['is_train'] == 'SI']

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.067331,-0.221432,-0.217879,0.646183,-0.507284,1.046258,0.896833,0.522917,0.581115,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1,-0.873616,0.442800,-0.072044,-0.063185,2.188279,0.154764,-0.395604,-0.572740,1.178257,-0.293098,...,0,0,0,0,1,0,0,0,1,0
2,0.067331,-0.088585,0.137197,0.646183,-0.507284,0.980221,0.848965,0.332854,0.098133,-0.293098,...,0,0,0,0,1,0,0,0,1,0
3,0.302568,-0.442842,-0.078385,0.646183,-0.507284,-1.859351,-0.682812,-0.572740,-0.494618,-0.293098,...,0,0,0,0,1,0,0,0,0,0
4,0.067331,0.619928,0.518903,1.355551,-0.507284,0.947203,0.753229,1.383791,0.469151,-0.293098,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.067331,-0.354278,-0.285470,-0.063185,-0.507284,0.914184,0.753229,-0.572740,-0.968819,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1456,-0.873616,0.664210,0.381311,-0.063185,0.391237,0.220801,0.178812,0.092480,0.765527,0.670556,...,0,0,0,0,1,0,0,0,1,0
1457,0.302568,-0.177150,-0.142806,0.646183,3.086800,-1.000876,1.040437,-0.572740,-0.365091,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1458,-0.873616,-0.088585,-0.057207,-0.772552,0.391237,-0.703711,0.561757,-0.572740,-0.861246,5.790336,...,0,0,0,0,1,0,0,0,1,0


In [23]:
train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.067331,-0.221432,-0.217879,0.646183,-0.507284,1.046258,0.896833,0.522917,0.581115,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1,-0.873616,0.442800,-0.072044,-0.063185,2.188279,0.154764,-0.395604,-0.572740,1.178257,-0.293098,...,0,0,0,0,1,0,0,0,1,0
2,0.067331,-0.088585,0.137197,0.646183,-0.507284,0.980221,0.848965,0.332854,0.098133,-0.293098,...,0,0,0,0,1,0,0,0,1,0
3,0.302568,-0.442842,-0.078385,0.646183,-0.507284,-1.859351,-0.682812,-0.572740,-0.494618,-0.293098,...,0,0,0,0,1,0,0,0,0,0
4,0.067331,0.619928,0.518903,1.355551,-0.507284,0.947203,0.753229,1.383791,0.469151,-0.293098,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.067331,-0.354278,-0.285470,-0.063185,-0.507284,0.914184,0.753229,-0.572740,-0.968819,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1456,-0.873616,0.664210,0.381311,-0.063185,0.391237,0.220801,0.178812,0.092480,0.765527,0.670556,...,0,0,0,0,1,0,0,0,1,0
1457,0.302568,-0.177150,-0.142806,0.646183,3.086800,-1.000876,1.040437,-0.572740,-0.365091,-0.293098,...,0,0,0,0,1,0,0,0,1,0
1458,-0.873616,-0.088585,-0.057207,-0.772552,0.391237,-0.703711,0.561757,-0.572740,-0.861246,5.790336,...,0,0,0,0,1,0,0,0,1,0


In [24]:
y_train

0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1460, dtype: float64

In [25]:
# Define the cross-validation strategy
cv = KFold(n_splits=10, shuffle=True, random_state=124345)

In [26]:
from sklearn.compose import make_column_selector

cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)

In [27]:
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_linear_processor = OneHotEncoder(handle_unknown="ignore")
num_linear_processor = StandardScaler()

linear_preprocessor = make_column_transformer(
    (num_linear_processor, num_selector), (cat_linear_processor, cat_selector)
)
linear_preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001CA829E0160>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001CA829E05E0>)])

In [28]:
cat_tree_processor = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1,
    encoded_missing_value=-2,
)

num_tree_processor = SimpleImputer(strategy="mean", add_indicator=True)

tree_preprocessor = make_column_transformer(
    (num_tree_processor, num_selector), (cat_tree_processor, cat_selector)
)
tree_preprocessor

ColumnTransformer(transformers=[('simpleimputer',
                                 SimpleImputer(add_indicator=True),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001CA829E0160>),
                                ('ordinalencoder',
                                 OrdinalEncoder(encoded_missing_value=-2,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001CA829E05E0>)])

Mostramos las 50 variables más importantes de RandomForest

In [29]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd

# Entrenar el modelo de RandomForest
rf = RandomForestRegressor(criterion='squared_error', max_depth=20, random_state=42)
rf.fit(X_train, y_train)

# Encontrar las 50 características más importantes
feature_importances = rf.feature_importances_
features = X_train.columns
importance_df = pd.DataFrame({'feature': features, 'importance': feature_importances})
top_50_RandomForest = importance_df.sort_values(by='importance', ascending=False).head(50)['feature']


In [30]:
top_50_RandomForest

3               OverallQual
15                GrLivArea
25               GarageCars
11              TotalBsmtSF
12                 1stFlrSF
8                BsmtFinSF1
5                 YearBuilt
187            CentralAir_Y
2                   LotArea
4               OverallCond
1               LotFrontage
6              YearRemodAdd
13                 2ndFlrSF
10                BsmtUnfSF
24              GarageYrBlt
23               Fireplaces
27              OpenPorchSF
38              MSZoning_RM
205       GarageType_Detchd
33                   MoSold
26               WoodDeckSF
7                MasVnrArea
160             BsmtQual_Gd
0                MSSubClass
201       GarageType_Attchd
22             TotRmsAbvGrd
28            EnclosedPorch
20             BedroomAbvGr
207        GarageFinish_Unf
34                   YrSold
18                 FullBath
42             LotShape_Reg
211           GarageQual_TA
21             KitchenAbvGr
215           GarageCond_TA
193          Kitchen

In [32]:
top_45_RandomForest = top_50_RandomForest.iloc[5:]
top_45_RandomForest

8                BsmtFinSF1
5                 YearBuilt
187            CentralAir_Y
2                   LotArea
4               OverallCond
1               LotFrontage
6              YearRemodAdd
13                 2ndFlrSF
10                BsmtUnfSF
24              GarageYrBlt
23               Fireplaces
27              OpenPorchSF
38              MSZoning_RM
205       GarageType_Detchd
33                   MoSold
26               WoodDeckSF
7                MasVnrArea
160             BsmtQual_Gd
0                MSSubClass
201       GarageType_Attchd
22             TotRmsAbvGrd
28            EnclosedPorch
20             BedroomAbvGr
207        GarageFinish_Unf
34                   YrSold
18                 FullBath
42             LotShape_Reg
211           GarageQual_TA
21             KitchenAbvGr
215           GarageCond_TA
193          KitchenQual_Gd
150            ExterCond_Fa
16             BsmtFullBath
61      Neighborhood_IDOTRR
229    SaleCondition_Normal
19                 H

In [33]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Clase para seleccionar las características más importantes
class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.feature_names = feature_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.feature_names]

# Crear la pipeline
rf_pipeline = Pipeline([
    ('feature_selector', TopFeatureSelector(top_45_RandomForest)),
    ('random_forest', RandomForestRegressor(criterion='squared_error', max_depth=20, random_state=42))
])

rf_pipeline


Pipeline(steps=[('feature_selector',
                 TopFeatureSelector(feature_names=8                BsmtFinSF1
5                 YearBuilt
187            CentralAir_Y
2                   LotArea
4               OverallCond
1               LotFrontage
6              YearRemodAdd
13                 2ndFlrSF
10                BsmtUnfSF
24              GarageYrBlt
23               Fireplaces
27              OpenPorchSF
38              MSZoning_RM
205       GarageType_Detchd
33                   MoSold
26               WoodDeckSF
7                MasVnrArea
160             BsmtQual_Gd
0                MSSubClass
201       GarageType_Attchd
22             TotRmsAbvGrd
28            EnclosedPorch
20             B...
193          KitchenQual_Gd
150            ExterCond_Fa
16             BsmtFullBath
61      Neighborhood_IDOTRR
229    SaleCondition_Normal
19                 HalfBath
194          KitchenQual_TA
59     Neighborhood_Edwards
37              MSZoning_RL
217            PavedDrive_Y
58     Neighborhood_Crawfor
167         BsmtExposure_No
165         BsmtExposure_Gd
45          LandContour_Lvl
141     Exterior2nd_VinylSd
Name: feature, dtype: object)),
                ('random_forest',
                 RandomForestRegressor(max_depth=20, random_state=42))])

In [34]:
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# KNN pipeline
knn_pipeline = make_pipeline(KNeighborsRegressor())

knn_pipeline

Pipeline(steps=[('kneighborsregressor', KNeighborsRegressor())])

In [35]:
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd

# Entrenar el modelo de GradientBoosting
gb = GradientBoostingRegressor(random_state=42, learning_rate=0.15, max_depth=4, n_estimators=333)
gb.fit(X_train, y_train)

# Encontrar las 50 características más importantes
feature_importances = gb.feature_importances_
features = X_train.columns
importance_df = pd.DataFrame({'feature': features, 'importance': feature_importances})
top_50_GradientBoost = importance_df.sort_values(by='importance', ascending=False).head(50)['feature']
top_50_GradientBoost

3               OverallQual
15                GrLivArea
11              TotalBsmtSF
25               GarageCars
8                BsmtFinSF1
5                 YearBuilt
6              YearRemodAdd
12                 1stFlrSF
4               OverallCond
2                   LotArea
23               Fireplaces
187            CentralAir_Y
13                 2ndFlrSF
38              MSZoning_RM
1               LotFrontage
24              GarageYrBlt
194          KitchenQual_TA
205       GarageType_Detchd
18                 FullBath
10                BsmtUnfSF
27              OpenPorchSF
201       GarageType_Attchd
21             KitchenAbvGr
215           GarageCond_TA
61      Neighborhood_IDOTRR
58     Neighborhood_Crawfor
26               WoodDeckSF
165         BsmtExposure_Gd
16             BsmtFullBath
28            EnclosedPorch
30              ScreenPorch
33                   MoSold
228    SaleCondition_Family
149            ExterQual_TA
217            PavedDrive_Y
19                 H

In [36]:
top_45_GradientBoost = top_50_GradientBoost.iloc[5:]
top_45_GradientBoost

5                 YearBuilt
6              YearRemodAdd
12                 1stFlrSF
4               OverallCond
2                   LotArea
23               Fireplaces
187            CentralAir_Y
13                 2ndFlrSF
38              MSZoning_RM
1               LotFrontage
24              GarageYrBlt
194          KitchenQual_TA
205       GarageType_Detchd
18                 FullBath
10                BsmtUnfSF
27              OpenPorchSF
201       GarageType_Attchd
21             KitchenAbvGr
215           GarageCond_TA
61      Neighborhood_IDOTRR
58     Neighborhood_Crawfor
26               WoodDeckSF
165         BsmtExposure_Gd
16             BsmtFullBath
28            EnclosedPorch
30              ScreenPorch
33                   MoSold
228    SaleCondition_Family
149            ExterQual_TA
217            PavedDrive_Y
19                 HalfBath
34                   YrSold
229    SaleCondition_Normal
37              MSZoning_RL
211           GarageQual_TA
117     Exterior1st_

In [37]:
gradient_boosting_pipeline = Pipeline([
    ('feature_selector', TopFeatureSelector(top_50_GradientBoost)),
    ('gradient_boosting', GradientBoostingRegressor(random_state=42, learning_rate=0.15, max_depth=4, n_estimators=333))
])

gradient_boosting_pipeline

Pipeline(steps=[('feature_selector',
                 TopFeatureSelector(feature_names=3               OverallQual
15                GrLivArea
11              TotalBsmtSF
25               GarageCars
8                BsmtFinSF1
5                 YearBuilt
6              YearRemodAdd
12                 1stFlrSF
4               OverallCond
2                   LotArea
23               Fireplaces
187            CentralAir_Y
13                 2ndFlrSF
38              MSZoning_RM
1               LotFrontage
24              GarageYrBlt
194          KitchenQual_TA
205       GarageType_Detchd
18                 FullBath
10                BsmtUnfSF
27              OpenPorchSF
201       GarageType_Attchd
21             K...
229    SaleCondition_Normal
37              MSZoning_RL
211           GarageQual_TA
117     Exterior1st_BrkFace
22             TotRmsAbvGrd
7                MasVnrArea
169        BsmtFinType1_GLQ
59     Neighborhood_Edwards
223            SaleType_New
200          Functional_Typ
195         Functional_Maj2
20             BedroomAbvGr
78          Condition1_Norm
Name: feature, dtype: object)),
                ('gradient_boosting',
                 GradientBoostingRegressor(learning_rate=0.15, max_depth=4,
                                           n_estimators=333,
                                           random_state=42))])

In [38]:
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd

# Entrenar el modelo de GradientBoosting
xgb = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, gamma=0.05, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.5, n_estimators=3000,
                             reg_alpha=0.5, reg_lambda=0.8,
                             subsample=0.5,
                             random_state =42, seed=42)
xgb.fit(X_train, y_train)

# Encontrar las 50 características más importantes
feature_importances = xgb.feature_importances_
features = X_train.columns
importance_df = pd.DataFrame({'feature': features, 'importance': feature_importances})
top_50_XGBoost = importance_df.sort_values(by='importance', ascending=False).head(50)['feature']
top_50_XGBoost

149            ExterQual_TA
3               OverallQual
25               GarageCars
18                 FullBath
194          KitchenQual_TA
15                GrLivArea
211           GarageQual_TA
23               Fireplaces
6              YearRemodAdd
5                 YearBuilt
37              MSZoning_RL
201       GarageType_Attchd
187            CentralAir_Y
11              TotalBsmtSF
161             BsmtQual_TA
169        BsmtFinType1_GLQ
217            PavedDrive_Y
12                 1stFlrSF
13                 2ndFlrSF
215           GarageCond_TA
64       Neighborhood_NAmes
8                BsmtFinSF1
38              MSZoning_RM
2                   LotArea
22             TotRmsAbvGrd
16             BsmtFullBath
165         BsmtExposure_Gd
4               OverallCond
21             KitchenAbvGr
1               LotFrontage
31                 PoolArea
151            ExterCond_Gd
125      Exterior1st_Stucco
172        BsmtFinType1_Unf
20             BedroomAbvGr
77         Condition

In [39]:
top_45_XGBoost = top_50_XGBoost.iloc[5:]
top_45_XGBoost

15                GrLivArea
211           GarageQual_TA
23               Fireplaces
6              YearRemodAdd
5                 YearBuilt
37              MSZoning_RL
201       GarageType_Attchd
187            CentralAir_Y
11              TotalBsmtSF
161             BsmtQual_TA
169        BsmtFinType1_GLQ
217            PavedDrive_Y
12                 1stFlrSF
13                 2ndFlrSF
215           GarageCond_TA
64       Neighborhood_NAmes
8                BsmtFinSF1
38              MSZoning_RM
2                   LotArea
22             TotRmsAbvGrd
16             BsmtFullBath
165         BsmtExposure_Gd
4               OverallCond
21             KitchenAbvGr
1               LotFrontage
31                 PoolArea
151            ExterCond_Gd
125      Exterior1st_Stucco
172        BsmtFinType1_Unf
20             BedroomAbvGr
77         Condition1_Feedr
191        Electrical_SBrkr
69     Neighborhood_OldTown
42             LotShape_Reg
80          Condition1_PosN
150            Exter

In [43]:
# XGBoost pipeline

xgboost_pipeline = Pipeline([
    ('feature_selector', TopFeatureSelector(top_45_XGBoost)),
    ('xgboost', XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, gamma=0.05, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.5, n_estimators=3000,
                             reg_alpha=0.5, reg_lambda=0.8,
                             subsample=0.5,
                             random_state =42, seed=42))
])

xgboost_pipeline

Pipeline(steps=[('feature_selector',
                 TopFeatureSelector(feature_names=15                GrLivArea
211           GarageQual_TA
23               Fireplaces
6              YearRemodAdd
5                 YearBuilt
37              MSZoning_RL
201       GarageType_Attchd
187            CentralAir_Y
11              TotalBsmtSF
161             BsmtQual_TA
169        BsmtFinType1_GLQ
217            PavedDrive_Y
12                 1stFlrSF
13                 2ndFlrSF
215           GarageCond_TA
64       Neighborhood_NAmes
8                BsmtFinSF1
38              MSZoning_RM
2                   LotArea
22             TotRmsAbvGrd
16             BsmtFu...
                              feature_types=None, gamma=0.05, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=1.5, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=3000, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [44]:
from sklearn.linear_model import LassoCV

lasso_pipeline = make_pipeline(LassoCV(max_iter=3000))
lasso_pipeline

Pipeline(steps=[('lassocv', LassoCV(max_iter=3000))])

In [45]:
import lightgbm as lgb

# Entrenar el modelo de GradientBoosting
lb = lgb.LGBMRegressor(objective='regression', verbose=-1, bagging_fraction = 0.8, bagging_seed=9, seed=42, n_estimators=300, max_depth=4, learning_rate=0.15)

lb.fit(X_train, y_train)

# Encontrar las 50 características más importantes
feature_importances = lb.feature_importances_
features = X_train.columns
importance_df = pd.DataFrame({'feature': features, 'importance': feature_importances})
top_50_LightGBM = importance_df.sort_values(by='importance', ascending=False).head(50)['feature']
top_50_LightGBM

15                GrLivArea
2                   LotArea
24              GarageYrBlt
11              TotalBsmtSF
1               LotFrontage
12                 1stFlrSF
10                BsmtUnfSF
8                BsmtFinSF1
3               OverallQual
5                 YearBuilt
4               OverallCond
28            EnclosedPorch
27              OpenPorchSF
7                MasVnrArea
6              YearRemodAdd
33                   MoSold
26               WoodDeckSF
13                 2ndFlrSF
34                   YrSold
229    SaleCondition_Normal
25               GarageCars
78          Condition1_Norm
9                BsmtFinSF2
0                MSSubClass
23               Fireplaces
20             BedroomAbvGr
16             BsmtFullBath
22             TotRmsAbvGrd
61      Neighborhood_IDOTRR
30              ScreenPorch
187            CentralAir_Y
127     Exterior1st_Wd Sdng
38              MSZoning_RM
59     Neighborhood_Edwards
211           GarageQual_TA
186            Heati

In [46]:
top_45_LightGBM = top_50_LightGBM.iloc[5:]
top_45_LightGBM

12                 1stFlrSF
10                BsmtUnfSF
8                BsmtFinSF1
3               OverallQual
5                 YearBuilt
4               OverallCond
28            EnclosedPorch
27              OpenPorchSF
7                MasVnrArea
6              YearRemodAdd
33                   MoSold
26               WoodDeckSF
13                 2ndFlrSF
34                   YrSold
229    SaleCondition_Normal
25               GarageCars
78          Condition1_Norm
9                BsmtFinSF2
0                MSSubClass
23               Fireplaces
20             BedroomAbvGr
16             BsmtFullBath
22             TotRmsAbvGrd
61      Neighborhood_IDOTRR
30              ScreenPorch
187            CentralAir_Y
127     Exterior1st_Wd Sdng
38              MSZoning_RM
59     Neighborhood_Edwards
211           GarageQual_TA
186            HeatingQC_TA
74     Neighborhood_StoneBr
201       GarageType_Attchd
58     Neighborhood_Crawfor
200          Functional_Typ
117     Exterior1st_

In [49]:
import lightgbm as lgb

# LightGBM pipeline
lightgbm_pipeline = Pipeline([
    ('feature_selector', TopFeatureSelector(top_45_LightGBM)),
    ('lightgbm', lgb.LGBMRegressor(objective='regression', verbose=-1, bagging_fraction = 0.8, bagging_seed=9, seed=42, n_estimators=300, max_depth=4, learning_rate=0.15))
])

lightgbm_pipeline

Pipeline(steps=[('feature_selector',
                 TopFeatureSelector(feature_names=12                 1stFlrSF
10                BsmtUnfSF
8                BsmtFinSF1
3               OverallQual
5                 YearBuilt
4               OverallCond
28            EnclosedPorch
27              OpenPorchSF
7                MasVnrArea
6              YearRemodAdd
33                   MoSold
26               WoodDeckSF
13                 2ndFlrSF
34                   YrSold
229    SaleCondition_Normal
25               GarageCars
78          Condition1_Norm
9                BsmtFinSF2
0                MSSubClass
23               Fireplaces
20             BedroomAbvGr
16             BsmtFullBath
22             TotRmsAbv...
117     Exterior1st_BrkFace
165         BsmtExposure_Gd
161             BsmtQual_TA
160             BsmtQual_Gd
55     Neighborhood_BrkSide
167         BsmtExposure_No
50         LotConfig_Inside
45          LandContour_Lvl
19                 HalfBath
48            LotConfig_FR2
Name: feature, dtype: object)),
                ('lightgbm',
                 LGBMRegressor(bagging_fraction=0.8, bagging_seed=9,
                               learning_rate=0.15, max_depth=4,
                               n_estimators=300, objective='regression',
                               seed=42, verbose=-1))])

In [48]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
import numpy as np
import catboost as cb



# Entrenar el modelo de GradientBoosting
cat = cb.CatBoostRegressor(
        iterations=500, 
        depth=6, 
        learning_rate=0.05, 
        loss_function='RMSE', 
        verbose=False, 
        random_seed=42)

cat.fit(X_train, y_train)

# Encontrar las 50 características más importantes
feature_importances = cat.feature_importances_
features = X_train.columns
importance_df = pd.DataFrame({'feature': features, 'importance': feature_importances})
top_50_cat = importance_df.sort_values(by='importance', ascending=False).head(50)['feature']
top_50_cat


3               OverallQual
15                GrLivArea
11              TotalBsmtSF
25               GarageCars
6              YearRemodAdd
4               OverallCond
2                   LotArea
12                 1stFlrSF
23               Fireplaces
8                BsmtFinSF1
5                 YearBuilt
149            ExterQual_TA
13                 2ndFlrSF
187            CentralAir_Y
24              GarageYrBlt
19                 HalfBath
194          KitchenQual_TA
160             BsmtQual_Gd
193          KitchenQual_Gd
229    SaleCondition_Normal
215           GarageCond_TA
27              OpenPorchSF
26               WoodDeckSF
1               LotFrontage
37              MSZoning_RL
145         MasVnrType_None
20             BedroomAbvGr
217            PavedDrive_Y
16             BsmtFullBath
211           GarageQual_TA
22             TotRmsAbvGrd
38              MSZoning_RM
78          Condition1_Norm
201       GarageType_Attchd
58     Neighborhood_Crawfor
18                 F

In [50]:
import catboost as cb

# CatBoost pipeline
catboost_pipeline = Pipeline([
    ('feature_selector', TopFeatureSelector(top_50_cat)),
    ('lightgbm',cb.CatBoostRegressor(
        iterations=500, 
        depth=6, 
        learning_rate=0.05, 
        loss_function='RMSE', 
        verbose=False, 
        random_seed=42
    ))
])


catboost_pipeline

Pipeline(steps=[('feature_selector',
                 TopFeatureSelector(feature_names=3               OverallQual
15                GrLivArea
11              TotalBsmtSF
25               GarageCars
6              YearRemodAdd
4               OverallCond
2                   LotArea
12                 1stFlrSF
23               Fireplaces
8                BsmtFinSF1
5                 YearBuilt
149            ExterQual_TA
13                 2ndFlrSF
187            CentralAir_Y
24              GarageYrBlt
19                 HalfBath
194          KitchenQual_TA
160             BsmtQual_Gd
193          KitchenQual_Gd
229    SaleCondition_Normal
215           GarageCond_TA
27              OpenP...
201       GarageType_Attchd
58     Neighborhood_Crawfor
18                 FullBath
10                BsmtUnfSF
165         BsmtExposure_Gd
0                MSSubClass
33                   MoSold
34                   YrSold
155        Foundation_PConc
28            EnclosedPorch
42             LotShape_Reg
59     Neighborhood_Edwards
223            SaleType_New
97        HouseStyle_1Story
200          Functional_Typ
30              ScreenPorch
207        GarageFinish_Unf
Name: feature, dtype: object)),
                ('lightgbm',
                 <catboost.core.CatBoostRegressor object at 0x000001CA8BEBAB50>)])

In [51]:
from sklearn.svm import SVR

svr_pipeline = make_pipeline(
    SVR(C=10, kernel='rbf', gamma='scale', epsilon=0.1)
)

svr_pipeline

Pipeline(steps=[('svr', SVR(C=10))])

In [52]:
# Define the base models
base_models = [
    ('random_forest', rf_pipeline),
    ('knn', knn_pipeline),
    ('gradient_boosting', gradient_boosting_pipeline),
    ('xgboost', xgboost_pipeline),
    ('lasso', lasso_pipeline),
    ('lightgbm', lightgbm_pipeline),
    ('catboost', catboost_pipeline),
    ('SVR', svr_pipeline)
    
]

stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=Ridge())
stacking_regressor

StackingRegressor(estimators=[('random_forest',
                               Pipeline(steps=[('feature_selector',
                                                TopFeatureSelector(feature_names=8                BsmtFinSF1
5                 YearBuilt
187            CentralAir_Y
2                   LotArea
4               OverallCond
1               LotFrontage
6              YearRemodAdd
13                 2ndFlrSF
10                BsmtUnfSF
24              GarageYrBlt
23               Fireplaces
27              OpenPorchSF
38              MSZoning_RM
205       GarageType_Detchd
33                   MoSold
26               WoodDeckSF
7                MasVnrArea
160             BsmtQual_Gd
0                MSSubClass
201       Ga...
0                MSSubClass
33                   MoSold
34                   YrSold
155        Foundation_PConc
28            EnclosedPorch
42             LotShape_Reg
59     Neighborhood_Edwards
223            SaleType_New
97        HouseStyle_1Story
200          Functional_Typ
30              ScreenPorch
207        GarageFinish_Unf
Name: feature, dtype: object)),
                                               ('lightgbm',
                                                <catboost.core.CatBoostRegressor object at 0x000001CA8BEBAB50>)])),
                              ('SVR', Pipeline(steps=[('svr', SVR(C=10))]))],
                  final_estimator=Ridge())

In [53]:
# Validate each base model using RMSE through cross-validation
for name, model in base_models:
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse_scores = np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=kfold))
    print(f'RMSE for {name}: {rmse_scores.mean()}')
    


RMSE for random_forest: 0.16099571388004758
RMSE for knn: 0.17229283285271046
RMSE for gradient_boosting: 0.12978472747401146
RMSE for xgboost: 0.13266666191879242
RMSE for lasso: 0.13989552407260733
RMSE for lightgbm: 0.1339017011851118
RMSE for catboost: 0.12111687017023218
RMSE for SVR: 0.1449105501650159


In [54]:
from sklearn.metrics import mean_squared_error

# Validate the stacking regressor using RMSLE through cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
rmsle_scores = np.sqrt(-cross_val_score(stacking_regressor, X_train, y_train, scoring="neg_mean_squared_error", cv=kfold))
print(f'RMSLE for Stacking Regressor: {rmsle_scores.mean()}')

# Train the stacking regressor on the entire training data
stacking_regressor.fit(X_train, y_train)

# Display RMSLE for the stacking model on the training data
y_pred_stacking = stacking_regressor.predict(X_train)
rmsle_stacking = np.sqrt(mean_squared_error(y_train, y_pred_stacking))
print(f'RMSLE for Stacking Regressor on training data: {rmsle_stacking}')

RMSLE for Stacking Regressor: 0.1210557536849383
RMSLE for Stacking Regressor on training data: 0.055744702331061796


In [55]:
X_test = X_test.drop('SalePrice', axis=1)

In [56]:
pred = stacking_regressor.predict(X_test)

In [57]:
salida = pd.DataFrame({'Id': test_ids, 'SalePrice': np.expm1(pred)})
salida.to_csv("Prueba15.csv", index=False)